<b style = 'color:red'>Must read:</b> **Hands-on Programming with R (PART III.8: S3)**    
"file:///C:/Books/R/Hands-On%20Programming%20with%20R%20(%20PDFDrive%20).pdf"

# Introduction

Values are not the only place to store information in R, and functions are not the only
way to create unique behavior. You can also do both of these things with R’s S3 system.

The S3 system provides a simple way to create object-specific behavior in R. In other
words, it is R’s version of object-oriented programming (OOP). The system is imple‐
mented by generic functions. These functions examine the class attribute of their input
and call a class-specific method to generate output. Many S3 methods will look for and
use additional information that is stored in an object’s attributes. Many common R
functions are S3 generics.
R’s S3 system is more helpful for the tasks of computer science than the tasks of data
science, but understanding S3 can help you troubleshoot your work in R as a data
scientist.

# The S3 system

<b style = 'color:red'>The S3 system allows R functions to behave in different ways for different classes</b>

S3 refers to a class system built into R. The system governs how R handles objects of
different classes. Certain R functions will look up an object’s S3 class, and then behave
differently in response.

In [1]:
# The base::print function will behave differently for different class (e.g: POSIXct, numeric, ...)
print(15)

print(lubridate::ymd(011006))

[1] 15
[1] "2001-10-06"


R’s S3 system is built around three components: 
* attributes (especially the **`class`** at‐tribute)
* generic functions
* methods

# Attributes 

Many R objects come with attributes, pieces
of extra information that are given a name and appended to the object. Attributes do
not affect the values of the object, but stick to the object as a type of metadata that R can
use to handle the object

In [2]:
attributes(iris)

$names
[1] "Sepal.Length" "Sepal.Width"  "Petal.Length" "Petal.Width"  "Species"     

$class
[1] "data.frame"

$row.names
  [1]   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
 [19]  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
 [37]  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
 [55]  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
 [73]  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
 [91]  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
[109] 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
[127] 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
[145] 145 146 147 148 149 150

In [4]:
attributes(factor(letters))

$levels
 [1] "a" "b" "c" "d" "e" "f" "g" "h" "i" "j" "k" "l" "m" "n" "o" "p" "q" "r" "s"
[20] "t" "u" "v" "w" "x" "y" "z"

$class
[1] "factor"

R is very laissez faire when it comes to attributes. It will let you add any attributes that
you like to an object (and then it will usually ignore them). The only time R will complain
is when a function needs to find an attribute and it is not there.

>You can add any general attribute to an object with **`attr`**  
You can also use **`attr`** to lookup the value of any attribute of an object.

In [23]:
player <- list(name = 'VN Pikachu', level = 31, clan = 'VN Champions')

# set attribute
attr(player, 'game') <- 'Tank Force'

player

$name
[1] "VN Pikachu"

$level
[1] 31

$clan
[1] "VN Champions"

attr(,"game")
[1] "Tank Force"

In [10]:
# get the value of attribute 'game'
attr(player, 'game')

[1] "Tank Force"

In [11]:
# show all attributes
attributes(player)

$names
[1] "name"  "level" "clan" 

$game
[1] "Tank Force"

>You can set multiple attributes in one step with **`structure`**. This will return a new object.

In [21]:
stat <- c(9011, 61023)

# set multiple attributes: game, country
stat_with_attr <- structure(stat, game = 'tank force', country = 'Russian')

stat_with_attr

[1]  9011 61023
attr(,"game")
[1] "tank force"
attr(,"country")
[1] "Russian"

In [22]:
attributes(stat_with_attr)

$game
[1] "tank force"

$country
[1] "Russian"

# Generic Functions

Genetic functions are functions that do different things in different cases. For example **`print`** is a *generic function*. For each different class (numeric, string, POSIXct, POSIXlt, ...) it will print in a different format.

In [24]:
print(5)

[1] 5


In [25]:
print('Hello World!')

[1] "Hello World!"


In [27]:
print(lubridate::make_date(2001, 10, 06))

[1] "2001-10-06"


# Methods

When you call `print`, `print` call a special function `UseMethod`:

In [29]:
print

function (x, ...) 
UseMethod("print")
<bytecode: 0x00000000415a3db8>
<environment: namespace:base>

In [30]:
UseMethod

function (generic, object)  .Primitive("UseMethod")

`UseMethod` examines the class of the input that you provide for the first argument of
`print`, and then passes all of your arguments to a new function designed to handle that
class of input. For example, when you give print a POSIXct object, UseMethod will pass
all of print’s arguments to `print.POSIXct`. R will then run `print.POSIXct` and return
the results.

In [31]:
print.POSIXct

function (x, tz = "", usetz = TRUE, max = NULL, ...) 
{
    if (is.null(max)) 
        max <- getOption("max.print", 9999L)
    FORM <- if (missing(tz)) 
        function(z) format(z, usetz = usetz)
    else function(z) format(z, tz = tz, usetz = usetz)
    if (max < length(x)) {
        print(FORM(x[seq_len(max)]), max = max + 1, ...)
        cat(" [ reached 'max' / getOption(\"max.print\") -- omitted", 
            length(x) - max, "entries ]\n")
    }
    else if (length(x)) 
        print(FORM(x), max = max, ...)
    else cat(class(x)[1L], "of length 0\n")
    invisible(x)
}
<bytecode: 0x0000000004ce6660>
<environment: namespace:base>

If you give print a factor object, UseMethod will pass all of print’s arguments to
`print.factor`. R will then run `print.factor` and return the results:

In [32]:
print.factor

function (x, quote = FALSE, max.levels = NULL, width = getOption("width"), 
    ...) 
{
    ord <- is.ordered(x)
    if (length(x) == 0L) 
        cat(if (ord) 
            "ordered"
        else "factor", "(0)\n", sep = "")
    else {
        xx <- character(length(x))
        xx[] <- as.character(x)
        keepAttrs <- setdiff(names(attributes(x)), c("levels", 
            "class"))
        attributes(xx)[keepAttrs] <- attributes(x)[keepAttrs]
        print(xx, quote = quote, ...)
    }
    maxl <- if (is.null(max.levels)) 
        TRUE
    else max.levels
    if (maxl) {
        n <- length(lev <- encodeString(levels(x), quote = ifelse(quote, 
            "\"", "")))
        colsep <- if (ord) 
            " < "
        else " "
        T0 <- "Levels: "
        if (is.logical(maxl)) 
            maxl <- {
                width <- width - (nchar(T0, "w") + 3L + 1L + 
                  3L)
                lenl <- cumsum(nchar(lev, "w") + nchar(colsep, 
                  "w"))
                if (n <= 1L || lenl[n] <= width) 
                  n
                else max(1L, which.max(lenl > width) - 1L)
            }
        drop <- n > maxl
        cat(if (drop) 
            paste(format(n), ""), T0, paste(if (drop) 
            c(lev[1L:max(1, maxl - 1)], "...", if (maxl > 1) lev[n])
        else lev, collapse = colsep), "\n", sep = "")
    }
    if (!isTRUE(val <- .valid.factor(x))) 
        warning(val)
    invisible(x)
}
<bytecode: 0x00000000054a7980>
<environment: namespace:base>

`print.POSIXct` and `print.factor` are called methods of print. 

>You can see which methods exist for a generic function by calling **`methods`** on the function.

In [33]:
# print has almost 200 methods (which gives you an idea of how many classes exist in R):
methods(print)

  [1] print.acf*                                          
  [2] print.AES*                                          
  [3] print.anova*                                        
  [4] print.aov*                                          
  [5] print.aovlist*                                      
  [6] print.ar*                                           
  [7] print.Arima*                                        
  [8] print.arima0*                                       
  [9] print.AsIs                                          
 [10] print.aspell*                                       
 [11] print.aspell_inspect_context*                       
 [12] print.bibentry*                                     
 [13] print.Bibtex*                                       
 [14] print.browseVignettes*                              
 [15] print.by                                            
 [16] print.bytes*                                        
 [17] print.changedFiles*                               

In [35]:
methods(head)

[1] head.data.frame* head.default*    head.ftable*     head.function*  
[5] head.matrix      head.table*     
see '?methods' for accessing help and source code

In [37]:
methods(summary)

 [1] summary.aov                    summary.aovlist*              
 [3] summary.aspell*                summary.check_packages_in_dir*
 [5] summary.connection             summary.data.frame            
 [7] summary.Date                   summary.default               
 [9] summary.Duration*              summary.ecdf*                 
[11] summary.factor                 summary.glm                   
[13] summary.infl*                  summary.Interval*             
[15] summary.lm                     summary.loess*                
[17] summary.manova                 summary.matrix                
[19] summary.mlm*                   summary.nls*                  
[21] summary.packageStatus*         summary.Period*               
[23] summary.POSIXct                summary.POSIXlt               
[25] summary.ppr*                   summary.prcomp*               
[27] summary.princomp*              summary.proc_time             
[29] summary.srcfile                summary.srcref            

### Method dispatch

To write a method for a *generic function* to handle a specific class, use:  
```r
generic_function_name.class_name <- function(...) {...}
```

Let's create a class named `allool`:

In [39]:
player <- list(name = 'Hadi', level = 35)

class(player) <- 'allool'

Writing a method for the function `print` that handle class named `allool`:

In [50]:
# generic_function_name: print
# class_name: allool

print.allool = function(x, ...) stringr::str_c('xXx-', x$name, '-xXx(', x$level, ')')

In [51]:
print(player)

[1] "xXx-Hadi-xXx(35)"

In [52]:
# remove method `print.allool` from generic function `print`
rm(print.allool)

If you give print an object whose class or classes do not have a print method, `UseMethod` will call `print.default`, a special method written to handle general cases.

In [54]:
# we remove print method for class `allool`, so in this case, `UseMethod` will call `print.default`
print(player)

$name
[1] "Hadi"

$level
[1] 35

attr(,"class")
[1] "allool"


# Classes

You can use the S3 system to make a robust new class of objects in R. Then R will treat
objects of your class in a consistent, sensible manner. To make a class:
1. Choose a name for your class.
2. Assign each instance of your class a `class` attribute.
3. Write class methods for any generic function likely to use objects of your class

In [58]:
# every methods written for class factor
# Notice that methods will not be able to show you methods that come in an unloaded R package:
methods(class = 'factor')

 [1] [             [[            [[<-          [<-           all.equal    
 [6] as.character  as.data.frame as.Date       as.list       as.logical   
[11] as.POSIXlt    as.vector     coerce        Compare       droplevels   
[16] format        initialize    is.na<-       length<-      levels<-     
[21] Math          Ops           plot          print         relevel      
[26] relist        rep           show          slotsFromS3   summary      
[31] Summary       xtfrm        
see '?methods' for accessing help and source code

# Further information

Read the rest of the chapter(S3 and debugging, S4 and S5, ...) 